
# Buenas Practicas

Habilita el  `type checking` in VSCODE. Si no lo hace automaticamente, manualmente abriendo la paleta de comandos(CTRL+SHIFT+P), escribe `Preferences: Open Workspace Settings (JSON)`, dale ENTER, añade o modifica la configuracion de Pylance: `"python.analysis.typeCheckingMode": "basic"`


# Primeros Pasos: Configuración del Entorno de Base de Datos con Docker y MongoDB Compass

Para construir los cimientos de nuestra aplicación, hemos configurado un entorno de base de datos robusto utilizando `Docker` para la gestión de contenedores y `MongoDB Compass` como nuestra interfaz gráfica para interactuar con la base de datos.

1. Despliegue de la Base de Datos MongoDB con Docker

```sh
    docker run --name tuTiendaDB --detach --mount src=dbAppTienda,dst=/data/db --publish 27017:27017 mongo:latest
```

`--publish 27017:27017:` El primer 27017 se refiere al puerto de la máquina HOST, en mi caso mi WSL; El segundo 27017 se refiere al puerto interno del contenedor donde MongoDB está escuchando las conexiones. Sin este, no sera posible intercatur desde `MongoDB Compass`

2. Acceso al Contenedor para Interacción Directa (Shell)

```sh
    docker exec -it tuTiendaDB bash
```

Una vez dentro del SHELL del contenedor, usar el comando `mongosh`, que es la nueva interfaz de línea de comandos para MongoDB. 

3. Conexión a la Base de Datos con MongoDB Compass.

`MongoDB Compass` es nuestra herramienta gráfica preferida para gestionar y visualizar los datos en MongoDB. Dada la exposición del puerto 27017, conectar Compass es sencillo. Por ahora. 


## Contenerización de la API con FastAPI y Docker

Para el desarrollo y despliegue de nuestra API basada en FastAPI, adoptaremos un enfoque de contenerización utilizando Docker. A continuación, se presenta el Dockerfile que define los pasos para construir la imagen de nuestra API

```DOCKERFILE
FROM alpine:latest

RUN apk add --update python3

RUN apk add py3-pip

WORKDIR /web-server

COPY ["./requirements.txt", "."]

RUN pip install -r requirements.txt --break-system-packages

COPY ["./main.py", "."]

CMD ["python3", "./main.py"]

```

### Analisando el DockerFile

- ``CMD ["python3", "./main.py"]``: Define el comando predeterminado que se ejecutará cuando se inicie un contenedor a partir de esta imagen. En este caso, instruye al contenedor a ejecutar main.py utilizando el intérprete de python3.

### Construyendo la imagen Docker

```sh
docker build --tag fastapi-app-image .
```

``--tag fastapi-app-image`` : Asigna la etiqueta fastapi-app-image a la imagen resultante, facilitando su identificación y gestión.

### Ejecutando Un Contenedor basado en la Imagen

```sh
docker run --rm -it --name app-temporal fastapi-app-image 
```

Este comando inicia un nuevo contenedor utilizando la imagen ``fastapi-app-image``. Al no especificar un comando explícito al final de docker run, el contenedor ejecutará automáticamente la instrucción definida en el ``CMD`` del Dockerfile. En este caso, el CMD está configurado para ejecutar ``python3 ./main.py``, lo que resultará en la impresión del mensaje "hola mundo" definido en tu script main.py.

El parámetro ``--rm`` es crucial aquí: asegura que el contenedor sea eliminado automáticamente del sistema de Docker una vez que su proceso principal (en este caso, la ejecución de main.py) finalice. Esto es ideal para tareas que no requieren persistencia del contenedor, manteniendo tu entorno de Docker limpio.

### Ejercicio

Sobrescribir el Comando CMD para Acceder a una Shell Interactiva:

```sh
docker run --rm -it --name app-temporal fastapi-app-image /bin/sh
```
Al ejecutar este comando, se crea y lanza un nuevo contenedor a partir de la imagen *fastapi-app-image*. El argumento final ``/bin/sh`` sobrescribe la instrucción CMD especificada en el Dockerfile (que normalmente ejecutaría ``python3 ./main.py``). En lugar de iniciar la aplicación FastAPI, el contenedor iniciará directamente un proceso de shell (``/bin/sh``, el shell predeterminado en Alpine Linux).

## Mejorando el Proyecto para Mostrar unas Rutas de Prueba

Actualizaremos nuestro `main.py`, para tener una aplicacion FASTAPI basica con dos `endpoints`, uno en el root `/` y otro `/cat`, observa se configuraron funciones asincronas.

```py
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

# Create a FastAPI application instance
app = FastAPI()

# Define your first API endpoint (route)
@app.get("/", tags=['root'])
async def read_root_endpoint():
    nombre = 'Gustavo'
    html_content = f'<h1>Hello {nombre} From API, this is root</h1>'
    return HTMLResponse(html_content)


# Define your first API endpoint (route)
@app.get("/cat", tags=['cat'])
async def read_cat_endpoint():
    nombre = 'Cat'
    html_content = f'<h1>Hello {nombre} From API, this is CAT Endpoint</h1>'
    return HTMLResponse(html_content)
```



### Actualizando el DOCKERFILE

Dockerfile actualmente termina con `CMD ["python3", "./main.py"]`. Si bien esto funciona para un script Python simple, las aplicaciones FastAPI son servidas por un servidor ASGI como `Uvicorn`. Necesitamos decirle al CMD que ejecute Uvicorn, que a su vez ejecutará tu aplicación FastAPI (app).

```dockerfile
FROM alpine:latest

RUN apk add --update python3

RUN apk add py3-pip

WORKDIR /web-server

COPY ["./requirements.txt", "."]

RUN pip install -r requirements.txt --break-system-packages

COPY ["./main.py", "."]

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]  👈
```

- uvicorn: El comando para iniciar el servidor Uvicorn.

- main:app: Esto le dice a Uvicorn que busque una instancia de aplicación llamada app dentro del archivo main.py.

- --host 0.0.0.0: Esto es crucial para Docker. Le dice a Uvicorn que se enlace a todas las interfaces de red disponibles dentro del contenedor. Sin esto, tu aplicación solo sería accesible desde el propio contenedor, no desde tu máquina host u otros contenedores.

- --port 8000: Especifica que Uvicorn debe escuchar en el puerto 8000.

Vuelve a buildiar la imagen

### Ejecutando el Contenedor 

```sh
docker run --rm -it --publish 8000:8000  --name app-temporal fastapi-app-image
```

- -publish 8000:8000: Esto publica el puerto 8000 del contenedor al puerto 8000 de tu máquina host

Abre tu navegador en _http://localhost:8000_ y _http://localhost:8000/docs_ para ver la documentacion.